In [1]:
import scipy.io
import numpy as np 
import os 
import scipy.misc 
import matplotlib.pyplot as plt 
import tensorflow as tf
import cPickle as pickle
import pandas as pd
from scipy import ndimage
import json
import time
%matplotlib inline  
print ("Packages loaded")

Packages loaded


In [2]:
layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',
        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',
        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',
        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',
        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
)

def imread(path):
    return scipy.misc.imread(path).astype(np.float32)

def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),padding='SAME')
    return tf.nn.bias_add(conv, bias)

def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),padding='VALID')

def net(data_path):
    # load weights
    data = scipy.io.loadmat(data_path)
    weights = data['layers'][0]
    net = {}
    for i, name in enumerate(layers):
        kind = name[:4]
        if kind == 'conv':
            kernels, bias = weights[i][0][0][2][0]
            kernels = kernels.transpose(1, 0, 2, 3)
            bias = bias.reshape(-1)
            net[name] = {}
            net[name]['kernels'] = kernels
            net[name]['bias'] = bias
    return net

In [3]:
vgg_path = "./Model/imagenet-vgg-verydeep-19.mat"
vgg = net(vgg_path)
image_folder = "../../../Public/Image_Dataset/train2014_resized/"
image_list=os.listdir(image_folder)

In [4]:
len(image_list)

82783

In [5]:
# get vgg feature maps at conv5_3 (14*14*512)
def get_features(vgg, image_list, batch_size=500):
    iter_until = len(image_list) + batch_size 
    feature_vectors = np.ndarray((len(image_list), 14, 14, 512))
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.8
    with tf.Graph().as_default(), tf.Session(config=config) as sess:
        for start, end in zip(range(61500, iter_until, batch_size),range(61500 + batch_size, iter_until, batch_size)):
            image_batch_file = image_list[start:end]
            image_batch = np.array(map(lambda x: ndimage.imread(x, mode='RGB'), image_batch_file)).astype(np.float32)  
            current = image_batch
            for i, name in enumerate(layers):
                kind = name[:4]
                if name =='conv5_3':
                    print current.get_shape
                    start_time = time.time()
                    feature_vectors[start:end, :, :, :] = sess.run(current)
                    print("--- %s seconds ---" % (time.time() - start_time))
                    break
                elif kind == 'conv':
                    current = _conv_layer(current, vgg[name]['kernels'], vgg[name]['bias'])
                elif kind == 'relu':
                    current = tf.nn.relu(current)
                elif kind == 'pool':
                    current = _pool_layer(current)

            print str(start+batch_size) + " finish"
            
            # pickle file temporarily
            if end % 500 == 0:
                with open("feature_vectors" + str(end//500) + ".pickle", 'wb') as f:
                    pickle.dump(feature_vectors[end-500:end], f)
                    print("pickling success..")
                    f.close()

            elif end % 500== 283:
                with open("feature_vectors" + str(166) + ".pickle", 'wb') as f:
                    pickle.dump(feature_vectors[82500:end], f)
                    print("pickling success..")
                    f.close()
    return feature_vectors

In [6]:
annotation_path = '../../../Public/Image_Dataset/train2014_annotations/captions_train2014.json'
anno = json.loads(open(annotation_path).read())
image_ids=os.listdir(image_folder)
image_list = []
for i in range(len(image_ids)):
    image_list.append(anno['images'][i]['file_name'])
image_path = map(lambda x: image_folder + x, list(image_list))

In [7]:
print image_path[0:10]

[u'../../../Public/Image_Dataset/train2014_resized/COCO_train2014_000000057870.jpg', u'../../../Public/Image_Dataset/train2014_resized/COCO_train2014_000000384029.jpg', u'../../../Public/Image_Dataset/train2014_resized/COCO_train2014_000000222016.jpg', u'../../../Public/Image_Dataset/train2014_resized/COCO_train2014_000000520950.jpg', u'../../../Public/Image_Dataset/train2014_resized/COCO_train2014_000000069675.jpg', u'../../../Public/Image_Dataset/train2014_resized/COCO_train2014_000000547471.jpg', u'../../../Public/Image_Dataset/train2014_resized/COCO_train2014_000000122688.jpg', u'../../../Public/Image_Dataset/train2014_resized/COCO_train2014_000000392136.jpg', u'../../../Public/Image_Dataset/train2014_resized/COCO_train2014_000000398494.jpg', u'../../../Public/Image_Dataset/train2014_resized/COCO_train2014_000000090570.jpg']


In [8]:
len(image_path)

82783

In [9]:
all_feats = get_features(vgg, image_path)

<bound method Tensor.get_shape of <tf.Tensor 'Relu_13:0' shape=(500, 14, 14, 512) dtype=float32>>
--- 101.355960846 seconds ---
62000 finish
pickling success..
<bound method Tensor.get_shape of <tf.Tensor 'Relu_27:0' shape=(500, 14, 14, 512) dtype=float32>>
--- 91.0714039803 seconds ---
62500 finish
pickling success..
<bound method Tensor.get_shape of <tf.Tensor 'Relu_41:0' shape=(500, 14, 14, 512) dtype=float32>>
--- 93.2528018951 seconds ---
63000 finish
pickling success..
<bound method Tensor.get_shape of <tf.Tensor 'Relu_55:0' shape=(500, 14, 14, 512) dtype=float32>>
--- 87.9303359985 seconds ---
63500 finish
pickling success..
<bound method Tensor.get_shape of <tf.Tensor 'Relu_69:0' shape=(500, 14, 14, 512) dtype=float32>>
--- 89.1847009659 seconds ---
64000 finish
pickling success..
<bound method Tensor.get_shape of <tf.Tensor 'Relu_83:0' shape=(500, 14, 14, 512) dtype=float32>>
--- 88.6628570557 seconds ---
64500 finish
pickling success..
<bound method Tensor.get_shape of <tf.Te